## Data Preprocessing and Feature Enginnering

In [1]:
#packages needed

#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

#the module 'sys' allows istalling module from inside Jupyter
import sys

!{sys.executable} -m pip install numpy
import numpy as np 

!{sys.executable} -m pip install pandas
import pandas as pd

#Natrual Language ToolKit (NLTK)
!{sys.executable} -m pip install nltk
import nltk

#text normalization function
%run ./Text_Normalization_Function.ipynb

#ignore warnings about future changes in functions as they take too much space
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\j4a3m\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\j4a3m\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\j4a3m\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\j4a3m\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

In [2]:
#### Download Data
import pandas as pd
df = pd.read_csv('cleaned_data.csv')
df.groupby('overall').count().reviewText

overall
1     448
2     604
3    1498
4    3058
5    6276
Name: reviewText, dtype: int64

### Data undersampling and assigned to 3 class

In [128]:
# Random select 
## 3classes
import random

df_1 = []
df_3 = []
df_5 = []

for i in df.iterrows():
    if i[1]['overall'] == 1:
        i[1]['overall']= 'negative'
        df_1.append(i)
    elif i[1]['overall'] == 2:
        i[1]['overall']= 'negative'
        df_1.append(i)
    elif i[1]['overall'] == 3:
        i[1]['overall']= 'neutral'
        df_3.append(i)
    elif i[1]['overall'] == 4:
        i[1]['overall']= 'positive'
        df_5.append(i)
    elif i[1]['overall'] == 5:
        i[1]['overall']= 'positive'
        df_5.append(i)

## Undersampling and ranfom shuffle
from random import sample

df_1 = sample(df_1,1000)
df_3 = sample(df_3,1000)
df_5 = sample(df_5,1000)

combined_df = df_1+ df_5 + df_3
random.shuffle(combined_df)

## Append to new_df
new_df  = pd.DataFrame(columns = list(df.columns.values))
for i in range(len(combined_df)):
    new_df.loc[i] = combined_df[i][1].values
new_df.groupby('overall').count().reviewText


## Select through Ratings
df2 = new_df[['reviewText','overall']]

In [129]:
df2.groupby('overall').count().reviewText

overall
negative    1000
neutral     1000
positive    1000
Name: reviewText, dtype: int64

In [130]:
df2.overall

0        neutral
1       negative
2        neutral
3       positive
4       negative
          ...   
2995    positive
2996    positive
2997    negative
2998    negative
2999    negative
Name: overall, Length: 3000, dtype: object

## Vador + SVM

In [11]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\j4a3m\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [12]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [13]:
def convert_df(series):
    s = []
    for i in series.values:
        s.append(i)
    return s

In [121]:
def analyze_sentiment_vader_lexicon(review, threshold = 0.1, verbose = False):
    scores = analyzer.polarity_scores(review)  
    binary_sentiment = 'positive' if scores['compound'] >= threshold else 'negative'
    if verbose:                             
        print('VADER Polarity (Binary):', binary_sentiment)
        print('VADER Score:', round(scores['compound'], 2))
    return binary_sentiment,scores['compound']  

In [134]:
Reviews = convert_df(df2.reviewText)
Reviews_X = np.array(Reviews)

VADER_polarity_test = [analyze_sentiment_vader_lexicon(review, threshold=0.1) for review in Reviews_X]
VADER_polarity_test_df = pd.DataFrame(VADER_polarity_test, columns = ['predicted', 'VADER_Score'])
VADER_polarity_test_df['overall']= df2.overall
VADER_polarity_test_df = VADER_polarity_test_df.iloc[:, 1:]

In [135]:
VADER_polarity_test_df.groupby('overall').count().VADER_Score

overall
negative    1000
neutral     1000
positive    1000
Name: VADER_Score, dtype: int64

In [113]:
#VADER_polarity_test_df.overall = VADER_polarity_test_df.overall.astype(int)

In [161]:
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from scipy import stats


X_train, X_test, y_train, y_test = train_test_split( VADER_polarity_test_df['VADER_Score'], VADER_polarity_test_df.overall, train_size = 0.9)

model = linear_model.SGDClassifier(loss='hinge') 


X_train = np.array(X_train).reshape(-1, 1)
X_test = np.array(X_test).reshape(-1, 1)


model.fit(X_train, y_train)
predicted_model = model.predict(X_test) 
print('Accuracy rate:', np.round(metrics.accuracy_score(predicted_model, y_test), 3))

Accuracy rate: 0.387


In [162]:
y_test.value_counts()

neutral     117
negative    101
positive     82
Name: overall, dtype: int64

In [163]:
Vador = pd.DataFrame()
Vador['test'] = y_test
Vador['predict']= predicted_model

In [164]:
Final_result_Vador = pd.crosstab(pd.Series(Vador['test']), 
            pd.Series(Vador['predict']), 
            rownames = ['True:'], 
            colnames = ['Predicted:'], 
            margins = True)

Final_result_Vador

Predicted:,negative,positive,All
True:,,,
negative,50,51,101
neutral,37,80,117
positive,16,66,82
All,103,197,300


In [165]:
precision = 66/197
recall = 66/82
recall

0.8048780487804879

## PURE SVM

In [28]:
X_train, X_test, y_train, y_test = train_test_split( df2.iloc[:,:-1], df2.overall, train_size = 0.9)

In [29]:
def convert_df(series):
    s = []
    for i in series:
        s.append(i)
    return s

In [30]:
X_train = convert_df(X_train.reviewText)
X_test = convert_df(X_test.reviewText)

y_train = convert_df(y_train)
y_test = convert_df(y_test)

In [31]:
train_reviews_X = np.array(X_train)
train_polarity_y = np.array(y_train)

test_reviews_X = np.array(X_test)
test_polarity_y = np.array(y_test)
normalized_test_reviews_X = normalize_corpus(test_reviews_X)
normalized_train_reviews_X = normalize_corpus(train_reviews_X)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range = (1,2)) 
feature_matrix_TRAIN = vectorizer.fit_transform(normalized_train_reviews_X).astype(float)

In [33]:
feature_matrix_TRAIN_names = vectorizer.get_feature_names() 
feature_matrix_TRAIN_table = pd.DataFrame(data = feature_matrix_TRAIN.todense(), 
                                          columns = feature_matrix_TRAIN_names)
feature_matrix_TRAIN_table.head()

,01f,01f dp,01m,01m dp,09qcgt,09qcgt piece,0a,0a store,0i,0i trap,...,zone purpose,zone tomato,zoom,zoom fly,zoom inside,zooming,zooming bug,zucchini,zucchini plant,zucchini really
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
feature_matrix_TEST = vectorizer.transform(normalized_test_reviews_X)

In [35]:
from sklearn import linear_model

model = linear_model.SGDClassifier(loss='hinge', random_state = 0, penalty = 'l2') 
model.fit(feature_matrix_TRAIN, train_polarity_y)
predicted_model = model.predict(feature_matrix_TEST) 
print('Accuracy rate:', np.round(metrics.accuracy_score(test_polarity_y, predicted_model), 3))

Accuracy rate: 0.553


In [106]:
result = pd.DataFrame(data=test_polarity_y, columns = ['test'])
result['predict']=predicted_model

In [107]:
result

,test,predict
0,neutral,positive
1,negative,negative
2,neutral,neutral
3,negative,negative
4,negative,neutral
...,...,...
295,positive,positive
296,positive,negative
297,negative,positive
298,positive,positive


In [111]:
Final_result_SVM = pd.crosstab(pd.Series(result['test']), 
            pd.Series(result['predict']), 
            rownames = ['True:'], 
            colnames = ['Predicted:'], 
            margins = True)
Final_result_SVM



Predicted:,negative,neutral,positive,All
True:,,,,
negative,58,26,20,104
neutral,29,43,23,95
positive,11,25,65,101
All,98,94,108,300


In [110]:
precision = 65/108
recall = 65/101

0.6435643564356436

### Neural network

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as p
import keras

df=pd.read_csv("cleaned_data.csv",index_col=0)
df=df.loc[:,["reviewText","overall"]].dropna().reset_index(drop=True)

In [4]:
# Random select 
## 5classes
import random

df_1 = []
df_3 = []
df_5 = []

for i in df.iterrows():
    if i[1]['overall'] == 1:
        i[1]['overall']= 'negative'
        df_1.append(i)
    elif i[1]['overall'] == 2:
        i[1]['overall']= 'negative'
        df_1.append(i)
    elif i[1]['overall'] == 3:
        i[1]['overall']= 'neutral'
        df_3.append(i)
    elif i[1]['overall'] == 4:
        i[1]['overall']= 'positive'
        df_5.append(i)
    elif i[1]['overall'] == 5:
        i[1]['overall']= 'positive'
        df_5.append(i)

## Undersampling and ranfom shuffle
from random import sample

df_1 = sample(df_1,1000)
df_3 = sample(df_3, 1000)
df_5 = sample(df_5,1000)

combined_df = df_1+ df_5 + df_3
random.shuffle(combined_df)

## Append to new_df
new_df  = pd.DataFrame(columns = list(df.columns.values))
for i in range(len(combined_df)):
    new_df.loc[i] = combined_df[i][1].values
new_df.groupby('overall').count().reviewText


## Select through Ratings
df2 = new_df[['reviewText','overall']]

In [45]:
len(df2)

3000

In [5]:
from sklearn.model_selection import train_test_split
rev=df2["reviewText"]
target=df2["overall"]
x_train, x_test, y_train, y_test = train_test_split(rev,target,test_size=0.1,random_state=0,stratify=target)

In [6]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

max_features = 10000
maxlen = 10
batch_size = 32


tokenizer=Tokenizer(max_features,oov_token="<?>")
tokenizer.fit_on_texts(x_train)
X_train=tokenizer.texts_to_sequences(x_train)
X_test=tokenizer.texts_to_sequences(x_test)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')


print('Pad sequences (samples x text length)')
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

2700 train sequences
300 test sequences
Pad sequences (samples x text length)
X_train shape: (2700, 10)
X_test shape: (300, 10)


In [14]:
X_train_df

,0,1,2,3,4,5,6,7,8,9
0,87,5765,766,544,44,24,45,1110,44,145
1,3,232,5,1953,5,2344,1674,63,2,444
2,93,17,3314,46,2,266,49,13,2,272
3,392,978,746,269,27,17,13,2,549,1597
4,468,308,549,20,67,15,82,50,8,42
...,...,...,...,...,...,...,...,...,...,...
2695,21,7,135,52,131,16,419,4,65,151
2696,2023,10,169,77,15,144,16,1050,17,2658
2697,7,3770,11,59,241,3,448,198,13,70
2698,3,22,5439,5,124,1532,13,444,3,106


In [8]:
X_train_df = pd.DataFrame(X_train)
X_test_df = pd.DataFrame(X_test)
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)

In [87]:
del model

In [9]:
from keras import layers
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()

embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))

model.add(SimpleRNN(32, return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(16))
model.add(Dropout(0.2))
model.add(layers.Dense(3, activation='softmax')) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 64)            640000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 10, 32)            3104      
                                                                 
 dropout (Dropout)           (None, 10, 32)            0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 16)                784       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 3)                 51        
                                                                 
Total params: 643,939
Trainable params: 643,939
Non-trai

In [10]:
y_train = np.array(pd.get_dummies(pd.Series(y_train)))
y_test = np.array(pd.get_dummies(pd.Series(y_test)))


In [11]:
##　Grid Search for best output
import itertools
from itertools import product


Epochs = [300,350,400,450,500,550, 600, 650, 700]      # Grid search
Batch_size = [30,35,40,45,50, 55 ,65, 75, 85, 95, 100]
Comb = [ i for i in product(Epochs, Batch_size)]

In [12]:
def RNN(Epochs, Batch_size):
    model = Sequential()

    embedding_dim = 64
    model.add(Embedding(max_features, embedding_dim, input_length=maxlen))

    model.add(SimpleRNN(32, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(SimpleRNN(16))
    model.add(Dropout(0.2))
    model.add(layers.Dense(3, activation='softmax')) 
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_accuracy', verbose=1,
        save_best_only=True, mode='auto', period=1)



    history = model.fit(X_train, y_train,
                        epochs=Epochs,
                        batch_size = Batch_size,
                        callbacks=[checkpoint],
                        validation_data=(X_test,y_test))
    return history
    

In [ ]:
# Grid Search
Best_Accuracy = [0,(0,0)]
for combinations in Comb:
    Model = RNN(combinations[0], combinations[1])
    Accuracy = max(Model.history['val_accuracy'])
    if Best_Accuracy[0] < Accuracy:
        Best_Accuracy[0] = Accuracy
        Best_Accuracy[1] = combinations 
        del Model
    else:
        continue
        del Model
Best_Accuracy

In [97]:
Best_Accuracy

[0.6812346423333, (400, 45)]

In [13]:
history = RNN(400, 45)
from keras.models import load_model
true_model = load_model('best_model.hdf5')
pred = true_model.predict(np.array(X_test),steps=len(X_test))


Epoch 1/400
55/60 [==========================>...] - ETA: 0s - loss: 0.2272 - accuracy: 0.3455
Epoch 00001: val_accuracy improved from -inf to 0.31333, saving model to best_model.hdf5
60/60 [==============================] - 1s 9ms/step - loss: 0.2266 - accuracy: 0.3493 - val_loss: 0.2233 - val_accuracy: 0.3133
Epoch 2/400
52/60 [=========================>....] - ETA: 0s - loss: 0.1848 - accuracy: 0.5932
Epoch 00002: val_accuracy improved from 0.31333 to 0.37000, saving model to best_model.hdf5
60/60 [==============================] - 0s 6ms/step - loss: 0.1853 - accuracy: 0.5830 - val_loss: 0.2251 - val_accuracy: 0.3700
Epoch 3/400
52/60 [=========================>....] - ETA: 0s - loss: 0.1134 - accuracy: 0.7902
Epoch 00003: val_accuracy improved from 0.37000 to 0.38667, saving model to best_model.hdf5
60/60 [==============================] - 0s 6ms/step - loss: 0.1145 - accuracy: 0.7848 - val_loss: 0.2550 - val_accuracy: 0.3867
Epoch 4/400
53/60 [=========================>....] - ET

In [82]:
pred = true_model.predict(np.array(X_train),steps=len(X_train))

In [83]:
y = [i for i in y_train_df['overall']]
from sklearn.metrics import accuracy_score
Output = pd.DataFrame(pred, columns = ['negative','neutral','positive'])

score = []
for i in Output.iterrows():
    index = Output.columns[Output.iloc[i[0]] == max(i[1])][0]
    score.append(index)